# Assistive Technology (2023-03-24)

## Email System

`Install-Package MailKit`

In [15]:
#r "nuget:MailKit"
#r "nuget:DotNetEnv"

// Load .env file
DotNetEnv.Env.TraversePath().Load();

Installed Packages DotNetEnv, 2.5.0 MailKit, 3.6.0

## Recieveing Email

In [16]:
using MimeKit;
using MailKit;
using MailKit.Search;
using MailKit.Net.Imap;

using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate (DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    // The Inbox folder is always available on all IMAP servers...
    var inbox = client.Inbox;
    inbox.Open (FolderAccess.ReadOnly);

    Console.WriteLine ("Total messages: {0}", inbox.Count);
    Console.WriteLine ("Recent messages: {0}", inbox.Recent);

    for (int i = 0; i < inbox.Count; i++) {
        var message = inbox.GetMessage (i);
        Console.WriteLine ("Subject: {0}", message.Subject);
    }

    client.Disconnect (true);
}

Total messages: 3
Recent messages: 0
Subject: Your Google Account is live – now help your business grow
Subject: Recovery email verified for your Google Account
Subject: 2-Step Verification turned on
